In [1]:
# import des modules nécessaires
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from distinctipy import distinctipy
from matplotlib.font_manager import FontProperties
from matplotlib import rc
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import clear_output, display
from scipy.stats import chi2_contingency as chi2_contingency
import statsmodels.formula.api as statsapi
import statsmodels as stats
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# Modifier les paramètres d'affichage de la sortie des commandes dans Jupyter
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 100)

# import de la police Garamond
garamond_font = FontProperties(fname="C:/Windows/Fonts/GARA.TTF",size=14)
timesnewroman_font = FontProperties(fname="C:/Windows/Fonts/TIMES.TTF",size=14)

import des fichiers de données et des fichiers d'url pour récupérer les champs "Modifié le" et Notices associée".
Etant donné que les résultats de recherche ne peuvent pas contenir plus de 10000 enregistrements nous avons morcellé la recherche en 4 requêtes :
Il y a 4414 résultats qui correspondent à /France/Paris et Ile de France /Panthéon Sorbonne P1
Il y a 6335 résultats qui correspondent à /France/Paris et Ile de France /Tout saut Paris Panthéon Sorbonne P1
Il y a 7411 résultats qui correspondent à /France/Tout sauf Paris et Ile de France
Il y a 60 résultats qui correspondent à /Italie, Suisse et Non Renseignés
Soit un total de 18220 au lieu de 18217 résultats dans TRHAA.


In [ ]:
df_data_temp_6335 = pd.read_csv("input/TRHAA/DATA for Fr_ParisIDF_TtSaufsorbonne_6335.xlsx",header=0,sep=",", low_memory=False,encoding = "ISO-8859-1")
df_data_temp_4414 = pd.read_csv("input/TRHAA/DATA for Fr_ParisIDF_Sorbonne_4414.xlsx",header=0,sep=",", low_memory=False,encoding = "ISO-8859-1")
df_data_temp_7411 = pd.read_csv("input/TRHAA/DATA for Fr_TtSaufParis_7411.xlsx",header=0,sep=",", low_memory=False,encoding = "ISO-8859-1")
df_data_temp_60 = pd.read_csv("input/TRHAA/DATA for TtSaufFrance_60.xlsx",header=0,sep=",", low_memory=False,encoding = "ISO-8859-1")

df_url_temp_6335 = pd.read_csv("input/TRHAA/URL_List_Fr_ParisIDF_TtSaufSorbonne_6335.xlsx",header=0,sep=",", low_memory=False,encoding = "ISO-8859-1")
df_url_temp_4414 = pd.read_csv("input/TRHAA/URL_List_Fr_ParisIDF_Sorbonne_4414.xlsx",header=0,sep=",", low_memory=False,encoding = "ISO-8859-1")
df_url_temp_7411 = pd.read_csv("input/TRHAA/URL_List_Fr_TtSaufParis_7411.xlsx",header=0,sep=",", low_memory=False,encoding = "ISO-8859-1")
df_url_temp_60 = pd.read_csv("input/TRHAA/URL_List_TtSaufFrance_NonRenseigne_60.xlsx",header=0,sep=",", low_memory=False,encoding = "ISO-8859-1")


Croisement des dataframe DATA et URL sur la clé "URL de la page" pour avoir l'ensemble des données pour chaque

In [ ]:
# Merge des fichiers

# Column bind des logs et questionnaires
df_4414_temp = pd.merge(df_data_temp_4414, df_url_temp_4414,on=['Original_URL'],how='outer',suffixes=('','_url'))
df_6335_temp = pd.merge(df_data_temp_6335, df_url_temp_6335,on=['Original_URL'],how='outer',suffixes=('','_url'))
df_7411_temp = pd.merge(df_data_temp_7411, df_url_temp_7411,on=['Original_URL'],how='outer',suffixes=('','_url'))
df_60_temp = pd.merge(df_data_temp_60, df_url_temp_60,on=['Original_URL'],how='outer',suffixes=('','_url'))
print("Shape de df_4414_temp: ", df_4414_temp.shape)
print("shape de df_6335_temp: ", df_6335_temp.shape)
print("Shape de df_7411_temp: ", df_7411_temp.shape)
print("Shape de df_60_temp: ", df_60_temp.shape)

Le nombre de colonnes est identique entre tous les fichiers, le nombre de lignes correspond aux nombres prévus donc on peut concaténer l'ensemble des dataframes pour obtenir un seul dataframe.

In [ ]:
# Concaténation des dataframes

df_tmp_1et2 = pd.concat([df_4414_temp,df_6335_temp], ignore_index=True)
df_tmp_3et4=pd.concat([df_7411_temp,df_60_temp], ignore_index=True)
df_trhaa_initial = pd.concat([df_tmp_1et2,df_tmp_3et4], ignore_index=True)

# Vérification du format
print("Shape de df_tmp: ", df_trhaa_initial.shape)

Pré-traitement des données. Analyse des colonnes qui contiennent une valeur unique pour suppression du champ ultérieurement. A priori Illustration, RoleAuteurPrincipal

Comparaison des champs SoutenanceDateDebut et SoutenanceDateFin. Si identiques on n'en conserve qu'une seule.
Idem pour les champs TitreOuvrage et Titre


In [31]:
# Analyse colonnes si valeurs uniques
#print(df_trhaa_initial["Illustration"].groupby().count) # Champ Illustration
#print(df_trhaa_initial["RoleAuteurPrincipal"].groupby().count) # Champ RoleAuteurPrincipal

# copie de df_trhaa_initial en df_trhaa_tmp pour le traitement
df_trhaa_tmp = df_trhaa_initial

# Comparaison des champs SoutenanceDateDebut et Fin
df_comp1 = (df_trhaa_tmp["SoutenanceDateDebut"]==df_trhaa_tmp["SoutenanceDateFin"])
print(df_comp1.columns)
print(df_comp1[""].all())

if (df_comp1[""].all()) == "True":
    print("les colonnes SoutenanceDateDebut et SoutenanceDateFin sont égales")
else:
    print("elles sont différentes")

# Comparaison des champs TitreOuvrage et Titre
df_comp2 = (df_trhaa_tmp["TitreOuvrage"]==df_trhaa_tmp["Titre"])
print(df_comp2.columns)
print(df_comp2[""].all())

#if (df_comp2[""].all()) == "True":
#    print("les colonnes Titre et TitreOuvrage sont égales")
#else:
#    print("elles sont différentes")

# Nettoyage des champs inutiles pour obtenir df_trhaa_initial
#df_trhaa_initial_v2 = df_trhaa_initial.drop(['Illustration','RoleAuteurPrincipal','Titre','SoutenanceDateFin'])

# Renommage colonne SoutenanceDateDebut
#df_trhaa_initial_v2.rename(columns={'SoutenanceDateDebut':'DateSoutenance'})

False
